In [12]:
CNY_USD = 0.1389
RUB_USD = 0.010309087
CAD_USD = 0.74

Today = '29.02.2024'
Mounth = 'Feb'

Group = 'EuroChem'
# Group = 'SUEK'

In [13]:
# import os
# files_list = os.listdir('.')
# work_files_list = [x for x in files_list if ('.xlsx' in x and '~' not in x and 'BD' in x and Mounth in x)]
# Data_Debt = [x for x in work_files_list if 'Debt' in x][0]
# Data_Forwards = [x for x in work_files_list if 'Forwards' in x][0]
# Data_Swaps = [x for x in work_files_list if 'Swaps' in x][0]
# Sheet_in_data_file = 'Export Worksheet'

from datetime import date
Output_Debt = "_".join([str(date.today()), Group, 'quantum', 'Debt', f'{Mounth}.xlsx'])
Output_Deriv = "_".join([str(date.today()), Group, 'quantum', 'Deriv', f'{Mounth}.xlsx'])
Sheet_in_output_Deriv = 'Deriv'
Sheet_in_output_Debt = 'Debt'

Output_Debt

'2024-03-21_EuroChem_quantum_Debt_Feb.xlsx'

In [2]:
import oracledb
import pandas as pd

oracledb.init_oracle_client('C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Oracle\\instantclient_21_13')

connection = oracledb.connect(user="RISKCUSTOM", password="xxRiscRccess174!", host="exatest2-scan.moscow.eurochem.ru", port=1521, service_name='riskdev.moscow.eurochem.ru', disable_oob= True)

In [29]:
query = """select * from "RISKACCESS"."quantumDebt" where "reportDate" = TO_DATE('29/02/24', 'DD/MM/YY')"""
data_Debt_export = pd.read_sql(query, con=connection)
data_Debt_export.shape

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10388\3057460188.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_Debt_export = pd.read_sql(query, con=connection)


(118646, 73)

In [15]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period

Фильтрация данных

In [16]:
max_data = data_Debt_export.effectiveFrom.max() # Присваивание переменной значения мин даты в столбце effectiveFrom
data_Debt_work = data_Debt_export.query('holding == @Group & effectiveFrom == @max_data').reset_index(drop=True) # Фильтрация данных по столбцу holding (переменная Group) и effectiveFrom

data_Debt_work = data_Debt_work[~data_Debt_work.entity.isna()]
data_Debt_work.entity = data_Debt_work.entity.replace({'RUBND': 'RUSBO'})
data_Debt_work['Entity_group'] = merge_SalesUnits(data_Debt_work, col='entity', merge_col='ocpSegment').fillna('External')

Выполнение цикла для рассчета amount, amountUSD и присваивания значения counterparty для каждого DealSet

In [17]:
deals_data = data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] # Выбор нужного разреза данных
deals_data_Bonds = deals_data[deals_data.dealSet.str.contains('Bond')] # Фильтрация разреза по значениям в DealSet, которые содержат слово Bond

Bonds_list = deals_data_Bonds.dealSet.unique()

for Bond in Bonds_list:
    try:
        Bond_data = deals_data_Bonds.query('dealSet ==@Bond') # Отбор данных одного Bond
        assert len(Bond_data[Bond_data.instrumentOwner == 'EC_ISSUED_BONDS']) != 0

        for column in Bond_data[['amount', 'amountUSD']]:
            Traded_data_sum = Bond_data.query('instrumentOwner =="EC_TRADED_BONDS"')[column].sum()
            Issued_data = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"')[column]
            # Внесение нового значения amount в issued bonds
            Bond_data.loc[Bond_data['instrumentOwner'] =="EC_ISSUED_BONDS", column] = Traded_data_sum + Issued_data 
        # Внесение нового значения counterparty в traded bonds
        Issied_data_entity = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"').entity.squeeze()
        Bond_data.loc[Bond_data['instrumentOwner'] =="EC_TRADED_BONDS", 'counterparty'] = Issied_data_entity
        deals_data_Bonds.loc[deals_data_Bonds['dealSet'] == Bond] = Bond_data
    except:
        continue

deals_data.loc[deals_data.dealSet.str.contains('Bond')] = deals_data_Bonds
data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] = deals_data
# Запись выполненного цикла в в основные данные

In [25]:
check = data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']]
check = check[check.instrumentOwner.str.contains('BONDS')].reset_index(drop=True)
check[check.dealSet == check.dealSet.unique().tolist()[4]]

,entity,Entity_group,counterparty,amount,amountUSD,instrumentOwner,dealSet
52,RUEMO,RUFERT,NSD RU (EC),-1.000000e+10,-1.088504e+08,EC_ISSUED_BONDS,MCC RuBond 07 2020-04 R10b


In [26]:
data_Debt_work = Period(data_Debt_work, day_for_count=Today, col_with_date='actionDate').reset_index(drop=True)

data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5] 
# Обрезка " (EC)" в каждой строке

data_Debt_work = data_Debt_work[~data_Debt_work.counterparty.isna()]
data_Debt_work['CompCode'] = merge_Mapping(data_Debt_work, col='counterparty').fillna('External')
data_Debt_work['Counterparty_Group'] = merge_SalesUnits(data_Debt_work, col='CompCode', merge_col='ocpSegment').fillna('External')
# Merge counterparty с Mapping и SalesUnits

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:41: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  day = pd.to_datetime(day_for_count)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_10388\802095099.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5]


In [27]:
manual_map = data_Debt_work.loc[(data_Debt_work['cpartyType'] == 'Internal') & (data_Debt_work['CompCode'] == 'External'), ['counterparty', 'CompCode', 'Counterparty_Group']].drop_duplicates()
manual_map

manual_map2 = data_Debt_work.loc[data_Debt_work['Entity_group'] == 'External', ['entity', 'Entity_group']]
manual_map2

manual_map_print = pd.concat([manual_map, manual_map2], axis=1)
manual_map_print
# manual_map_print.to_excel('Manual_map.xlsx', index=False)

,counterparty,CompCode,Counterparty_Group,entity,Entity_group


In [9]:
# data_Debt_export.to_excel(Output_Debt, sheet_name = Sheet_in_output_Debt, index = False)
# Полные данные для проверки качества выполнения кода

Запись данных в файл Excel

In [28]:
data_Debt_work['Source'] = 'Quantum'

data_Debt_export_for_print = data_Debt_work[['entity', 'Entity_group','counterparty',\
                                               'Counterparty_Group','amount','eventCurrency','amountUSD','actionDate','Days',\
                                                'Period','dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]
# Отбор нужных столбцов

data_Debt_export_for_print = data_Debt_export_for_print.rename(columns={'amount': 'amoutn outstanding',\
                                                                        'eventCurrency': 'Currency',\
                                                                        'amountUSD': 'amount USD eq',\
                                                                        'actionDate': 'termEnd'})
# Переименование столбцов

data_Debt_export_for_print.to_excel(Output_Debt, sheet_name = Sheet_in_output_Debt, index = False)
# Запись в файл

Работа с Forwards и Swaps:

In [4]:
data_Forwards_export = pd.read_excel(Data_Forwards, sheet_name = Sheet_in_data_file)
data_Swaps_export = pd.read_excel(Data_Swaps, sheet_name = Sheet_in_data_file)

c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\KlimovaAnnaA\AppData\Local\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [70]:
data_Forwards_export['holdingEntity'] = data_Forwards_export['holdingEntity'].replace({'XXTSU': 'SUEK'})
data_Forwards_export = data_Forwards_export[data_Forwards_export['holdingEntity'] == Group].reset_index(drop=True)
data_Swaps_export['holdingEntity'] = data_Swaps_export['holdingEntity'].replace({'XXTSU': 'SUEK'})
data_Swaps_export = data_Swaps_export[data_Swaps_export['holdingEntity'] == Group].reset_index(drop=True)
# Знамена значений в стодбце holdingEntity и фильтр по Group

Проверка вхождений dealNo в Debt и остановка процесса в случае отсутвия данных:

In [71]:
Swap_dealNo_for_del_list = list(set(data_Swaps_export['dealNo']).intersection(set(data_Debt_export['dealNo'])))
Forward_dealNo_for_del_list = list(set(data_Forwards_export['dealNo']).intersection(set(data_Debt_export['dealNo'])))

data_Swaps_export = data_Swaps_export[~(data_Swaps_export.dealNo.isin(Swap_dealNo_for_del_list))]
data_Forwards_export = data_Forwards_export[~(data_Forwards_export.dealNo.isin(Forward_dealNo_for_del_list))]

assert ((len(data_Swaps_export) == 0) and (len(data_Forwards_export) == 0)) == False

In [75]:
Data_for_loop = data_Forwards_export[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]

for i in range(0, len(Data_for_loop)):
    row_list = list(Data_for_loop.loc[i])
    Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
    Data_for_loop.loc[i,'payFaceValue'] = Data_for_loop.loc[i,'payFaceValue'] * -1
Data_for_loop = Data_for_loop.sort_values('dealNo')
# цикл создания новых строк

Data_for_loop['Entity_group'] = merge_SalesUnits(Data_for_loop, col='entity', merge_col='ocpSegment').fillna('External')
# Merge столбца entity по mapping

Data_for_loop = Period(Data_for_loop, col_with_date='maturityDate', day_for_count=Today)
# Создание Days и Period

Data_for_loop['USD_equiv'] = 0
for i in range (0,len(Data_for_loop)):
    if Data_for_loop.loc[i, 'payFXCurrency'] == 'CNY':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * CNY_USD
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'RUB':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * RUB_USD
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'USD':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue']
    elif Data_for_loop.loc[i, 'payFXCurrency'] == 'CAD':
        Data_for_loop.loc[i, 'USD_equiv'] = Data_for_loop.loc[i, 'payFaceValue'] * CAD_USD
# создание USD_equiv

new_columns = ['Company', 'Settlement', 'Notional Amount (USD)', 'Unnamed', 'source']
Data_for_loop = Data_for_loop.reindex(columns=(Data_for_loop.columns.tolist() + new_columns))
Data_for_loop['source'] = 'Quantum'
# Создание пустых столбцов и столбца ресурса

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9520\659122996.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9520\659122996.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_9520\659122996.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

In [76]:
Data_for_loop_Swap = data_Swaps_export[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]

for i in range(0, len(Data_for_loop_Swap)):
    row_list = list(Data_for_loop_Swap.loc[i])
    Data_for_loop_Swap.loc[len(Data_for_loop_Swap)] = row_list[:4] + row_list[6:8] + row_list[6:]
    Data_for_loop_Swap.loc[i,'payFaceValue'] = Data_for_loop_Swap.loc[i,'payFaceValue'] * -1
Data_for_loop_Swap = Data_for_loop_Swap.sort_values('dealNo')
# цикл создания новых строк

Data_for_loop_Swap['Entity_group'] = merge_SalesUnits(Data_for_loop_Swap, col='entity', merge_col='ocpSegment').fillna('External')
# Merge столбца entity по mapping

Data_for_loop_Swap = Period(Data_for_loop_Swap, day_for_count=Today, col_with_date='maturityDate')
# Создание Days и Period

Data_for_loop_Swap['USD_equiv'] = 0
for i in range (0,len(Data_for_loop_Swap)):
    if Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'CNY':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue'] * CNY_USD
    elif Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'RUB':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue'] * RUB_USD
    elif Data_for_loop_Swap.loc[i, 'payFXCurrency'] == 'USD':
        Data_for_loop_Swap.loc[i, 'USD_equiv'] = Data_for_loop_Swap.loc[i, 'payFaceValue']
# создание USD_equiv

new_columns = ['Company', 'Settlement', 'Notional Amount (USD)', 'Unnamed', 'source']
Data_for_loop_Swap = Data_for_loop_Swap.reindex(columns=(Data_for_loop_Swap.columns.tolist() + new_columns))
Data_for_loop_Swap['source'] = 'Quantum'
# Создание пустых столбцов и столбца ресурса

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:44: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  


In [77]:
data = Data_for_loop[['entity','Entity_group','holdingEntity','Company',\
                    'counterparty','dealNo','payFXCurrency','payFaceValue',\
                    'USD_equiv','Settlement','Notional Amount (USD)',\
                    'Unnamed','maturityDate','Days','Period','source']]

data_Swap = Data_for_loop_Swap[['entity','Entity_group','holdingEntity','Company',\
                    'counterparty','dealNo','payFXCurrency','payFaceValue',\
                    'USD_equiv','Settlement','Notional Amount (USD)',\
                    'Unnamed','maturityDate','Days','Period','source']]
# Выбор столбцов

data_to_excel = pd.concat([data, data_Swap], axis=0)
# Объединение таблиц вниз

In [78]:
data_to_excel.loc[data_to_excel['Entity_group'] == 'External']

,entity,Entity_group,holdingEntity,Company,counterparty,dealNo,payFXCurrency,payFaceValue,USD_equiv,Settlement,Notional Amount (USD),Unnamed,maturityDate,Days,Period,source


In [79]:
data_to_excel.to_excel(Output_Deriv, sheet_name = Sheet_in_output_Deriv, index = False)
# Запись в файл 